# Read Before


*   Open From Github, run and commit do not change directly from colab notebook without group discussion
*   List item


# TODOS


*   List item
*   List item



# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Importing Necessary files

In [ ]:
import os
import random
from google.colab import files
import shutil
import xml.etree.ElementTree as ET
import os
import pandas as pd
from os import listdir
from os.path import join
import numpy as np
os.environ['KAGGLE_USERNAME'] = "khanfashee" # username from the json file
os.environ['KAGGLE_KEY'] = "97583e78a52b19db942cca0456cd0a66" # key from the json file
!pip -q uninstall -y kaggle
!pip -q install --upgrade pip
!pip -q install kaggle==1.5.6
!kaggle -v
!pip install PyYAML==5.3.1

     |████████████████████████████████| 2.1 MB 28.1 MB/s 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Kaggle API 1.5.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=594fdaf3e34902db6548af995d0112f5183c741a5ffdc8af5472d66d585c004e
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built PyYAML
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Downloading Datasets

In [ ]:
!kaggle datasets download -d khanfashee/dhaka-ai
!kaggle datasets download -d khanfashee/dhaka-ai-yaml-files

!unzip -q /content/dhaka-ai.zip
!unzip -q /content/dhaka-ai-yaml-files.zip

100% 1.39G/1.39G [00:13<00:00, 96.5MB/s]
100% 1.39G/1.39G [00:13<00:00, 111MB/s] 
  0% 0.00/4.25k [00:00<?, ?B/s]
100% 4.25k/4.25k [00:00<00:00, 3.93MB/s]


# Data PreProcessing For YOLOV5

## Renaming jpeg to jpg

In [ ]:
import os
os.rename(r'/content/train/Final Train Dataset/06.jpeg',r'/content/train/Final Train Dataset/06.jpg')
os.rename(r'/content/train/Final Train Dataset/65.jpeg',r'/content/train/Final Train Dataset/65.jpg')

## Corrupted Data

In [ ]:
os.remove('/content/train/Final Train Dataset/231.jpg')
os.remove('/content/train/Final Train Dataset/231.xml')
#os.remove('/content/train/Final Train Dataset/.ipynb_checkpoints.xml')


## Excluding All Png

In [ ]:
#!/usr/bin/python
from PIL import Image
import os, sys,glob

path = "/content/train/Final Train Dataset/"
dirs = os.listdir( path )
def resize():
      count=0
      for item in dirs:

        if os.path.isfile(path+item):
          if item.endswith(".jpg") or item.endswith(".png") or item.endswith(".jpeg") or item.endswith(".PNG") or item.endswith(".JPG") or item.endswith(".JPEG"):
            file_path=path+item
            if item.endswith(".png") or item.endswith(".PNG"):
              #os.rename(path+item,path+item[-3]+"jpg")
              #file_path=path+item[-3]+"jpg"
              os.remove(file_path)
              os.remove(file_path[:-3]+"xml")
              print(file_path)
              continue
            else:
              im = Image.open(file_path)
              f, e = os.path.splitext(file_path)
              os.remove(file_path)
              #imResize = im.resize((img_size,img_size), Image.ANTIALIAS)
              imResize=im
              #im=im.convert('RGB')
              imResize.save(f+'.jpg' , 'JPEG', quality=100)
              count = count+1
            #print(count)



resize()

/content/train/Final Train Dataset/196.png
/content/train/Final Train Dataset/204.png
/content/train/Final Train Dataset/205.png
/content/train/Final Train Dataset/Pias (359).PNG
/content/train/Final Train Dataset/202.png
/content/train/Final Train Dataset/Pias (360).PNG
/content/train/Final Train Dataset/193.png
/content/train/Final Train Dataset/194.png
/content/train/Final Train Dataset/199.png
/content/train/Final Train Dataset/195.png
/content/train/Final Train Dataset/201.png
/content/train/Final Train Dataset/203.png
/content/train/Final Train Dataset/197.png
/content/train/Final Train Dataset/200.png


## Deleting Corrupted Files

In [ ]:
dirs = os.listdir( path )
def delete_files():
      count=0
      for item in dirs:

        if os.path.isfile(path+item):
          if item.endswith(".xml"):
            xml_file=open(path+item)
            tree=ET.parse(xml_file)
            root = tree.getroot()
            size = root.find('size')
            w = float(size.find('width').text)
            h = float(size.find('height').text)
            if (w==0 or h==0):
              os.remove(path+item)
              im_file=path+item
              im_file=im_file[:-3]+"jpg"
              os.remove(im_file)
delete_files()

## Generating Bounding Box And Train Val Split
<font color="red">

*   Augmentation Needs to be added
*   Random Seed validation needs to be added
*   Check Again if everything is correct in this block
</font>


In [ ]:
count = 0
classes = ['ambulance', 'army vehicle', 'auto rickshaw', 'bicycle', 'bus', 'car', 'garbagevan', 'human hauler', 'minibus', 'minivan', 'motorbike', 'pickup', 'policecar', 'rickshaw', 'scooter', 'suv', 'taxi', 'three wheelers (CNG)', 'truck', 'van', 'wheelbarrow'] # own data sets which classes which category to write, in the order

def convert(size, box):
    if (size[0]==0):

        x=0
        y=0
        w=0
        h=0

    else:
        dw = 1./(size[0])
        dh = 1./(size[1])
        x = (box[0] + box[1])/2.0 - 1
        y = (box[2] + box[3])/2.0 - 1
        w = box[1] - box[0]
        h = box[3] - box[2]
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
    return (x,y,w,h)

def convert_annotation(image_id):
    in_file = open('/content/train/Final Train Dataset/%s.xml'%(image_id), encoding = 'utf-8')

    out_file = open('/content/train/Final Train Dataset/%s.txt'%(image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = float(size.find('width').text)
    h = float(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        # bbox=np.zeros(4)

        # bbox[0]=(b[0]/w)*img_size
        # bbox[1]=(b[1]/w)*img_size
        # bbox[2]=(b[2]/h)*img_size
        # bbox[3]=(b[3]/h)*img_size


        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
        #print("====A Done======",image_id)

val_percent = 0.1
data_path = '/content/train/Final Train Dataset/' # darknet relative path folder
if not os.path.exists('/content/labels/'):
    os.makedirs('/content/labels/')
image_ids = [f for f in os.listdir ( '/content/train/Final Train Dataset')] # XML data storage folder
random.seed(42)
random.shuffle(image_ids)
train_file = open('/content/train.txt', 'w')
val_file = open('/content/val.txt', 'w')
for i, image_id in enumerate(image_ids):
         if image_id [-3:] == "xml":
            if i < (len(image_ids) * val_percent):
              val_file.write(data_path + '%s\n'%(image_id[:-3] + 'jpg'))
              print("Val:")
              print(image_id[:-4])
            else:
              train_file.write(data_path + '%s\n'%(image_id[:-3] + 'jpg'))
              print("Train:")
              print(image_id[:-4])
         convert_annotation(image_id[:-4])
train_file.close()
val_file.close()

Streaming output truncated to the last 5000 lines.
02
Train:
Numan_(76)
Train:
Pias (130)
Train:
Dipto_597
Train:
Numan_(326)
Train:
83
Train:
Dipto_582
Train:
Dipto_ 332
Train:
206
Train:
Dipto_466
Train:
Pias (210)
Train:
Numan_(155)
Train:
Pias (205)
Train:
Dipto_667
Train:
Dipto_ 158
Train:
Pias (233)
Train:
Dipto_772
Train:
Dipto_692
Train:
230
Train:
Navid_664
Train:
Dipto_792
Train:
226
Train:
Pias (292)
Train:
Dipto_762
Train:
Numan_(112)
Train:
Pias (440)
Train:
54
Train:
Dipto_ 169
Train:
Navid_258
Train:
Numan_(412)
Train:
Numan_(199)
Train:
Dipto_ 100
Train:
Dipto_ 161
Train:
Navid_409
Train:
Dipto_ 73
Train:
Navid_262
Train:
Dipto_713
Train:
Navid_310
Train:
Numan_(384)
Train:
Pias (195)
Train:
Navid_457
Train:
Navid_719
Train:
Navid_419
Train:
Asraf_60
Train:
Pias (191)
Train:
136
Train:
Navid_313
Train:
Asraf_27
Train:
Navid_736
Train:
Navid_68
Train:
Navid_682
Train:
Numan_(434)
Train:
Navid_458
Train:
Numan_(36)
Train:
Navid_755
Train:
Dipto_836
Train:
Pias (432)
Train

## Moving Annotated test files to required directory

In [ ]:

if not os.path.exists('/content/val1'):
    os.makedirs('/content/val1')
dst = "/content/val1/"

with open('/content/val.txt') as my_file:
    for filename in my_file:
      #  src = os.path.join(filename) # .strip() to avoid un-wanted white spaces
        if os.path.isfile(filename[:-1]):
          shutil.move(filename[:-1], dst)
          shutil.move(filename[:-4]+"txt",dst)

if not os.path.exists('/content/train1'):
    os.makedirs('/content/train1')
dst = "/content/train1/"

with open('/content/train.txt') as my_file:
    for filename in my_file:
      #  src = os.path.join(filename) # .strip() to avoid un-wanted white spaces
      if os.path.isfile(filename[:-1]):
        shutil.move(filename[:-1], dst)
        shutil.move(filename[:-4]+"txt",dst)

# Getting Yolov5 From Git with specific commit

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git checkout 3b57cb56412976847b91ef184ca2f2a99491d8ab

Cloning into 'yolov5'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2683 (delta 0), reused 3 (delta 0), pack-reused 2679
Receiving objects: 100% (2683/2683), 5.44 MiB | 32.55 MiB/s, done.
Resolving deltas: 100% (1767/1767), done.
/content/yolov5
Note: checking out '3b57cb56412976847b91ef184ca2f2a99491d8ab'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 3b57cb5 Simplified inference (#1153)


In [ ]:
#!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install numpy==1.17

#!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

# Training
<font color="green">
Tune Here with various models,optimizer, epochs, starting weights(if any),
</font><br>
<font color="red">
<h1>Change Everytime the logdir it saves required files
</font>

In [ ]:
!python train.py --img 1024 --batch 4 --epochs 15 \
  --data /content/traffic_data.yaml --cfg /content/yolov5x.yaml \
  --name yolov5x_traffic \
  --logdir '/content/drive/My Drive/YoloV5DhakaAi/Exp3/' \
  --weights '/content/drive/My Drive/YoloV5DhakaAi/Exp2/exp5_yolov5x_traffic/weights/last.pt'

Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='/content/yolov5x.yaml', data='/content/traffic_data.yaml', device='', epochs=15, evolve=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[1024, 1024], local_rank=-1, logdir='/content/drive/My Drive/YoloV5DhakaAi/Exp3/', multi_scale=False, name='yolov5x_traffic', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=4, weights='/content/drive/My Drive/YoloV5DhakaAi/Exp2/exp5_yolov5x_traffic/weights/last.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir /content/drive/My Drive/YoloV5DhakaAi/Exp3/", view at http://localhost:6006/
2020-10-19 10:28:18.045104: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'lrf': 0.

# Detection on Test Set

In [ ]:
shutil.move('/content/detect3.py', '/content/yolov5/')

'/content/yolov5/detect3.py'

# <font color="green"> Tuning Can be done here by chosing different weights and conf threshold</font>
<font color="red">Do not change image size</font>

In [ ]:
if os.path.isfile("/content/pred_csv.txt"):
  os.remove("/content/pred_csv.txt")
!python detect3.py --weights '/content/drive/My Drive/YoloV5DhakaAi/Exp3/exp8_yolov5x_traffic/weights/last.pt' \
  --img 1024 --conf 0.10 --source /content/test1/test/ --save-txt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.1, device='', img_size=1024, iou_thres=0.45, output='inference/output', save_txt=True, source='/content/test1/test/', update=False, view_img=False, weights=['/content/drive/My Drive/YoloV5DhakaAi/Exp3/exp8_yolov5x_traffic/weights/last.pt'])
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

Fusing layers... 
Model Summary: 284 layers, 8.85252e+07 parameters, 0 gradients
image 1/500 /content/test1/test/Asraf_50_jpg.rf.7026694f0b9f37a6790982295c7e8663.jpg: 1024x1024 1 bicycles, 2 cars, 1 human haulers, 1 minivans, 4 motorbikes, 2 pickups, 2 trucks, 1 vans, Done. (0.105s)
image 2/500 /content/test1/test/Asraf_51_jpg.rf.0e3516baf7509bc2c4a4aa8deea494c2.jpg: 1024x1024 1 bicycles, 4 buss, 4 cars, 3 minivans, 3 motorbikes, 4 vans, 1 wheelbarrows, Done. (0.066s)
image 3/500 /content/test1/test/Asraf_52_jpg.rf.867869f276e6db3a09a84b99643df316.jpg: 1024x1024 2 buss, 1 cars, 1 minivans, 3 motorbi

# Generating And Downloading CSV

In [ ]:
temp_df=pd.read_csv('/content/pred_csv.txt',sep=",",header=None,names=["image_id","class","score", "xmin", "ymin","xmax","ymax"])
temp_df['image_id'] = temp_df['image_id'].str.replace("inference/output/", '')
temp_df['width']=1024
temp_df['height']=1024
# temp_df=temp_df

# temp_df.to_csv('submission.csv', index=False)

In [ ]:

temp_df=temp_df.sort_values(by=['xmin'])
# temp_df['score'] = 1.0
temp_df

,image_id,class,score,xmin,ymin,xmax,ymax,width,height
1421,Shykat_03_032_jpg.rf.81ed7c4cea1b4320230a275fe...,suv,0.38,0,606,242,1024,1024,1024
3866,sabiha(152)_jpg.rf.9b892eefb41c2860e3313ddb260...,minivan,0.18,0,501,41,606,1024,1024
166,Sabiha_(18)_jpg.rf.2d99bbe6021834328af969e718d...,suv,0.17,0,740,267,1024,1024,1024
3868,sabiha(152)_jpg.rf.9b892eefb41c2860e3313ddb260...,car,0.31,0,513,42,612,1024,1024
730,Sabiha_(8)_jpg.rf.24900113770665f45ff44a6bc6c3...,suv,0.14,0,838,97,1021,1024,1024
...,...,...,...,...,...,...,...,...,...
4846,sabiha(293)_jpg.rf.c77bb8cf2a094a91b11f90265cd...,car,0.49,1000,871,1024,925,1024,1024
474,Sabiha_(4)_jpg.rf.134cf55311b25147f3238699e22a...,three wheelers (CNG),0.13,1002,440,1024,540,1024,1024
1514,Shykat_4_ (2)_jpg.rf.467be143ff078021badd137f5...,car,0.13,1004,181,1024,266,1024,1024
4127,sabiha(194)_jpg.rf.c1223f37b6d87e8687e4e2ac7e9...,car,0.11,1006,366,1024,442,1024,1024


In [ ]:
temp_df.to_csv('submission_conf_.10_xmin_sorted.csv', index=False)